<h1> Identifying Driver Classes to Estimate the Safety Benefits of AI Semi-Autonomous Vehicle Technologies </h1>

<strong> Author: Micah Simmerman, Nolan Ollada, Nathan Palmer </strong>

<strong>Resource URL:</strong>

<strong>Database file(s), columnar dataset(s), etc.:</strong>

# Project Description
This project will investigate data mining techniques for efficient isolation of principle classes hidden inside massive datasets (i.e., those containing millions of data points or more). This project will identify a series of robust and sequential data mining techniques that can be used to extract, clean, normalize, and prepare the data for causal analysis. Our investigation will use Principal Component Analysis (PCA) and apriori-based Data Cube techniques to observe temporal and regional trends in automotive safety. We will use the insight derived from these Data Cubes to identify and predict the principal driver classes composing the SWITRS California Collision Data Set using Decision Tree Classification (DTC), Frequent Pattern Mining (FPM), and Bayesian Belief Networks (BBN). The project will conclude with a k-fold cross validation or/or cluster based data discovery  to estimate the accuracy and usefulness of each model.

In [1]:
# Libraries and packages
import os
import sqlite3
import pandas as pd
pd.__version__
import numpy as np
# from sklearn import datasets  # contains the iris data set, among other things.
from sklearn import decomposition, datasets
from sklearn.decomposition import PCA  # Super important for sklearn PCA
from sklearn.preprocessing import MinMaxScaler # For normalization
from sklearn.preprocessing import StandardScaler
from scipy import stats
from collections import Counter
import math
import seaborn as sns
import matplotlib.pyplot as plt

# SQLite db file connection is achieved in the try/except statements below. 
# TODO: add sections to read-in mySQL/PostGreSQL and csv files.

tables = []  # GLOBAL TABLE INDEX
attribute_list = []  # GLOBAL ATTRIBUTE INDEX

# NOTE: always store the switrs.sqlite file in the parent directory where this python notebook is kept.
db_file = '../switrs.sqlite'  # name the db file downloaded directly from https://www.kaggle.com/datasets/alexgude/california-traffic-collision-data-from-switrs
sqliteConnection = sqlite3.connect(db_file)

# if os.path.isfile(db_file):  # determine if there is any SQLite db file by that name exists in the specified file location.
sqliteConnection = sqlite3.connect(db_file)  # establish a connection if the file does exist.
cursor = sqliteConnection.cursor()  # create a cursor object.
table_query = 'SELECT name from sqlite_master where type= "table"'
cursor.execute(table_query)
result = cursor.fetchall()
print()
print(len(result), "tables found.")
for i in range(len(result)):
    tables.append(result[i][0])
    # print(result[i][0])

# the second part of the algorithm extracts the attributes of each table identified in the first step and places them into a 2D list. 
for table in tables:
    consumer_complaints_count_records = "PRAGMA table_info("
    consumer_complaints_count_records += str(table)  + ");"  # build the sqlite query string using the current list object
    cursor.execute(consumer_complaints_count_records)  # execute the query string
    result = cursor.fetchall()  # collect the results
    temp_list = []
    for item in result:
        temp_list.append(item[1])
    attribute_list.append(temp_list)  # export the list attribute table after it is built
    # print("The", table, "table contains", result[0][0], "data points")  # print the results
    # print()
print("Table and attribute name extraction completed successfully.")


print('DB file and Dependencies Loaded.')  
# else:
#     print(os.path.isfile(db_file), "No SQLite file detected.")  # otherwise, print the connection status.


4 tables found.
Table and attribute name extraction completed successfully.
DB file and Dependencies Loaded.


Start by collecting information about the table names in the database.

In [2]:
# Now we have a list of tables and their respective attribute columns.
print("Tables in the database: ", tables)
print("Attribute list: ", attribute_list)

Tables in the database:  ['case_ids', 'collisions', 'victims', 'parties']
Attribute list:  [['case_id', 'db_year'], ['case_id', 'population', 'county_city_location', 'county_location', 'primary_road', 'secondary_road', 'distance', 'direction', 'intersection', 'weather_1', 'weather_2', 'location_type', 'ramp_intersection', 'tow_away', 'collision_severity', 'killed_victims', 'injured_victims', 'party_count', 'primary_collision_factor', 'pcf_violation_code', 'pcf_violation_category', 'type_of_collision', 'motor_vehicle_involved_with', 'pedestrian_action', 'road_surface', 'road_condition_1', 'lighting', 'control_device', 'pedestrian_collision', 'bicycle_collision', 'motorcycle_collision', 'truck_collision', 'not_private_property', 'alcohol_involved', 'statewide_vehicle_type_at_fault', 'chp_vehicle_type_at_fault', 'severe_injury_count', 'other_visible_injury_count', 'complaint_of_pain_injury_count', 'pedestrian_killed_count', 'pedestrian_injured_count', 'bicyclist_killed_count', 'bicyclist_

## Data Cleaning and Preprocessing
run the code below to remove the attributes in the remove_cols_collisions and remove_cols_parties tables specified. You can run the above 2 code blocks again to see that the specified attributes were removed.

Running the following cell twice currently results in an error. 

    RATIONAL FOR ATTRIBUTE COLUMN REMOVAL:
    collisions.officer_id: responding officers identification is not relevant to the factors of a collision.
    collisions.chp_shift: chp_shift indicates the work shift period of the responding officer. More accurate temporal measurements exist.
    collisions.special_condition: 
    collisions.beat_type: 
    collisions.city_division_lapd: 
    collisions.chp_beat_class: 
    collisions.beat_number: 
    collisions.pcf_violation: 
    collisions.pcf_violation_subsection: 
    collisions.hit_and_run: 
    collisions.process_date: 

    For a complete list of attribute definitions, please visit: https://peteraldhous.com/Data/ca_traffic/SWITRS_codebook.pdf

# Warning:
The following cell block removes unwanted attribute columns from the SWITRS database file and will take a long time to execute. Repeated requests should not impose any harm on the database file. Time to complete last session = 14 minutes. 

In [3]:
# Increase the page count and turn off SQLite Journaling Mode to reduce the overhead of the SQLite ALTER TABLE statements.
increase_page_count = "PRAGMA max_page_count = 2147483646;"
turn_off_journal = "PRAGMA journal_mode=OFF"
cursor.execute(increase_page_count)
cursor.execute(turn_off_journal)  # turn off the journaling mode (temporarily) to avoid exceeding local disk space limitations.
sqliteConnection.commit()

In [4]:
# PLACE AN INDEX ON 'case_id' IN ALL THREE TABLES TO FACILITATE DRILL-THROUGH QUERIES. 
try:
    set_case_id_index_collisions = "CREATE INDEX collisions_case_id ON collisions(case_id);"
    set_case_id_index_parties = "CREATE INDEX parties_case_id ON parties(case_id);"
    set_case_id_index_victims = "CREATE INDEX victims_case_id ON victims(case_id);"
    cursor.execute(set_case_id_index_collisions)
    cursor.execute(set_case_id_index_parties)
    cursor.execute(set_case_id_index_victims)
    sqliteConnection.commit()
except:
    print("SWITRS DB file contains the following indices...") 
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("collisions table index 'collisions_case_id' created on 'case_id' attribute column.") 
    print("parties table index 'parties_case_id' created on 'case_id' attribute column.")
    print("victims table index 'victims_case_id' created on 'case_id' attribute column.")

SWITRS DB file contains the following indices...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
collisions table index 'collisions_case_id' created on 'case_id' attribute column.
parties table index 'parties_case_id' created on 'case_id' attribute column.
victims table index 'victims_case_id' created on 'case_id' attribute column.


In [5]:
# Include the attribute columns you wish to remove in the list(s) below.
remove_cols_collisions = ["side_of_highway", "road_condition_2", "primary_ramp", "secondary_ramp", "jurisdiction", "chp_beat_type", "reporting_district", "state_highway_indicator", "caltrans_county", "caltrans_district", "state_route", "route_suffix", "postmile_prefix", "postmile", "chp_road_type", "officer_id", "chp_shift", "special_condition", "beat_type", "city_division_lapd", "chp_beat_class", "beat_number", "pcf_violation", "pcf_violation_subsection", "hit_and_run", "process_date"]
remove_cols_parties = ["hazardous_materials", "oaf_violation_code"]
# Alternatively, we can identify the attribute ids with a string and concatenate it into a SQLite query statement.
# remove_cols_collisions = "officer_id, chp_shift, special_condition, beat_type, city_division_lapd, chp_beat_class, beat_number, pcf_violation, pcf_violation_subsection, hit_and_run, process_date"
# remove_cols_parties = "hazardous_materials, oaf_violation_code"

try:
    for col in remove_cols_collisions:
        drop_collisions_columns = "ALTER TABLE collisions DROP  "
        drop_collisions_columns += col  + ";" 
        cursor.execute(drop_collisions_columns)
        sqliteConnection.commit()
        print("Removed", col, "from the 'collisions' table.")
        print("working...")
    for attrib in remove_cols_parties:
        drop_parties_columns = "ALTER TABLE parties DROP "
        drop_parties_columns += attrib  + ";" 
        cursor.execute(drop_parties_columns)
        sqliteConnection.commit()
        print("Removed", attrib, "from the 'parties' table")
        print("working...")
    print("The identified attribute columns successfully removed from the switrs database file.")
    print("Overwrite ../switrs.sqlite with a newly decompressed switrs source file to reverse these changes")
except:
    print("These columns have been removed already.")

## CODE FOR RESTORING JOURNAL MODE (DEFAULT = OFF)
# turn_journal_on = "PRAGMA journal_mode=ON"  # let's leave this off for now. 
# cursor.execute(turn_journal_on)  # turn the journaling mode back on so we can roll-back our changes.
# sqliteConnection.commit()  # commit the change to the database file.
# print("journal mode is back on.")

These columns have been removed already.


In [6]:
# Now we have a list of tables and their respective attribute columns.+
for i in range(len(tables)):
    print(tables[i], "has attributes:", attribute_list[i])
    # print(attribute_list[i])
    

case_ids has attributes: ['case_id', 'db_year']
collisions has attributes: ['case_id', 'population', 'county_city_location', 'county_location', 'primary_road', 'secondary_road', 'distance', 'direction', 'intersection', 'weather_1', 'weather_2', 'location_type', 'ramp_intersection', 'tow_away', 'collision_severity', 'killed_victims', 'injured_victims', 'party_count', 'primary_collision_factor', 'pcf_violation_code', 'pcf_violation_category', 'type_of_collision', 'motor_vehicle_involved_with', 'pedestrian_action', 'road_surface', 'road_condition_1', 'lighting', 'control_device', 'pedestrian_collision', 'bicycle_collision', 'motorcycle_collision', 'truck_collision', 'not_private_property', 'alcohol_involved', 'statewide_vehicle_type_at_fault', 'chp_vehicle_type_at_fault', 'severe_injury_count', 'other_visible_injury_count', 'complaint_of_pain_injury_count', 'pedestrian_killed_count', 'pedestrian_injured_count', 'bicyclist_killed_count', 'bicyclist_injured_count', 'motorcyclist_killed_coun

In [7]:
# ENUMERATE DATA POINTS
data_point_count = 0
for table in range(len(tables)):
    count_records = 'SELECT COUNT(*) FROM ('  # start the string.
    count_records += tables[table] + ");"  # concatenate the variable and complete the string build.
    cursor.execute(count_records)  # 
    result = cursor.fetchall()
    data_point_count += result[0][0]
    print("The", tables[table], "table contains", result[0][0], "data points.")

print("Current number of data points accross tables of 'switrs.sqlite' database file =", data_point_count)

The case_ids table contains 9424334 data points.
The collisions table contains 9424334 data points.
The victims table contains 9639334 data points.
The parties table contains 18669166 data points.
Current number of data points accross tables of 'switrs.sqlite' database file = 47157168


### Training/Test Set Generation Using Random Data Point Sampling

Below, we create a Pandas DataFrame by pulling 10,000 samples at random from the switrs collisions table. We then extract these case_id's into a python list and use it to execute a set-based SQLite query to pull in the points from the other database tables. Doing this dramatically increases the performance of these queries. The representative data extracted from these columns is then used for data cleaning, nominal attribute quantification, and the application of normalizing functions (such as z-score) on numerical attribute columns to support common PCA libraries such as sklearn.

Note that the code blocks below can be used to triangulate on data points based on the multiple attributes you can examine in each representative dataframe. As an exmaple, suppose you wish to locate victims of a car accident who were fully-ejected from their vehicle as a result of failure to fasten a safety belt. To locate such an object, one might perform the following queries on the 'victims_matching_10000_training_df' data frame to examine a representative sample collected at random from the 'victims' switrs.sqlite database file. The following queries may be used to locate such a datapoint within the set. 

    # TAKE A LOOK AT A SPECIFIC COLUMN(S)
    print("BEGIN: Examine victims_matching_10000_training_df.loc[:,'victim_safety_equipment_2']")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(victims_matching_10000_training_df.loc[:,"victim_safety_equipment_2"])
    print()
    print("BEGIN: Examine victims_matching_10000_training_df.loc[:,'victim_ejected']")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(victims_matching_10000_training_df.loc[:,"victim_ejected"])

The code within this block populates the case_id list object with the case_id values collected in the random sample procedure. This list is then used to perform a set selection using a "SELECT FROM WHERE case_id IN ..." type SQLite query construction. The nature of set queries and generation of the index on the 'case_id' attribute column of each table makes this a very speedy query set. 

####  Generating the n=10,000 Training Sets (PANDAS data frames) Using Random Sampling w/o Replacement

The following code sections produce three (n=10,000) random sample dataframes, composed of matching data points from the 'collisions', 'parties', and 'victims' tables by using a 'case_id' sampling index, kept in the form of a python list 'n_10000_training_set_case_ids' in the cell below.

In [8]:
# Create a python list of the 'case_id' values for the n=10,000 random sample set that we are about to generate.
n_10000_training_set_case_ids = []  # this list will contain each 'case_id'

# The estimated completion time is 20.5 seconds.
print("Generating a n=10,000 random-sample training set from the 'collisions' table of the switrs.sqlite database file...")
print("#####################################################################################################")
# Read 10000 random samples from the collisions table into a local pandas DataFrame. (BOOTSTRAP SAMPLING)
collisions_n_10000_training_set_df = pd.read_sql_query("SELECT * FROM collisions ORDER BY RANDOM() LIMIT 10000;", sqliteConnection)
# sort the data frame in ascending order of case_id
collisions_n_10000_training_set_df.sort_values(by=['case_id'], ascending=True, kind='quicksort', na_position='last', ignore_index=False, key=None)# sort the df in row ascending order of case_id

print()
print("collisions_random_10000_df.head()...")
print(collisions_n_10000_training_set_df.head())
print(collisions_n_10000_training_set_df['case_id'].values[0])

# populate the case_ids list
for i in collisions_n_10000_training_set_df.index:
    n_10000_training_set_case_ids.append(collisions_n_10000_training_set_df['case_id'].values[i])

# Check our progress.
print("SAMPLE 'n_10000_training_set_case_ids':")
print(n_10000_training_set_case_ids)  # USE THIS LIST TO CALL THE DATA OBJECTS IN FROM THE OTHER TABLES (I.e., 'parties' and 'victims'). 

Generating a n=10,000 random-sample training set from the 'collisions' table of the switrs.sqlite database file...
#####################################################################################################

collisions_random_10000_df.head()...
    case_id        population county_city_location county_location  \
0   2584916    unincorporated                 5500        tuolumne   
1   1624184    unincorporated                 4300     santa clara   
2   3154565           >250000                 3711       san diego   
3  90321559    unincorporated                 3400      sacramento   
4  81218925  100000 to 250000                 1933     los angeles   

            primary_road     secondary_road  distance direction  intersection  \
0       SANGUINETTI LOOP            MONO WY     100.0     south           0.0   
1                 DAY RD            GERI LN      43.0      east           0.0   
2  RANCHO PENASQUITOS BL              RT 15      60.0      west           0.0   


In [9]:
# Estimated cell runtime: 0.5 seconds
## This cell runs lightning fast (approximately 4 seconds) because of its' set-based query and the index placed on case_id in each table.
## Here, we are building a tuple-formatted string containing the case_id's we are interested in collecting from switrs. We close the argument at the end of the paragraph.
training_set_case_ids = "("  # we will close this when we append the other argument strings.
for i in n_10000_training_set_case_ids:  # scan through the training set case_id's.
    training_set_case_ids += i  # append each case_id we find to the sql query string.
    training_set_case_ids += ", "  # delete this substring before closing the string.
training_set_case_ids = training_set_case_ids[:-2]  # 
training_set_case_ids += ") ORDER BY case_id DESC;"

# Complete each of the sql queries
parties_query = "SELECT * FROM parties WHERE case_id IN " + training_set_case_ids
victims_query = "SELECT * FROM victims WHERE case_id IN " + training_set_case_ids

# PULL IN THE CORRESPONDING RECORDS FROM 'parties' and 'victims' TABLES
parties_matching_10000_training_df = pd.read_sql_query(parties_query, sqliteConnection)
victims_matching_10000_training_df = pd.read_sql_query(victims_query, sqliteConnection)

# Check our work
print("Generating a n=10,000 random-sample training set from the 'parties' table of the switrs.sqlite database file...")
print("#######################################################")
print(".head() view of the 'parties_matching_10000_training_df' dataframe object:")
print(parties_matching_10000_training_df.head())
print("END")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print()
print("Generating a n=10,000 random-sample training set from the 'victims' table of the switrs.sqlite database file...")
print("#######################################################")
print(".head() view of the 'victims_matching_10000_training_df' dataframe object:")
print(victims_matching_10000_training_df.head())
print("END")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print()

Generating a n=10,000 random-sample training set from the 'parties' table of the switrs.sqlite database file...
#######################################################
.head() view of the 'parties_matching_10000_training_df' dataframe object:
         id  case_id  party_number      party_type  at_fault party_sex  \
0  10488854  9266131             2  parked vehicle         0      None   
1  10488853  9266131             1          driver         1      male   
2  10488378  9265869             1          driver         1    female   
3  10487938  9265562             2          driver         0    female   
4  10487937  9265562             1          driver         1      male   

   party_age                      party_sobriety party_drug_physical  \
0        NaN                      not applicable      not applicable   
1       22.0  had been drinking, under influence                None   
2       23.0  had been drinking, under influence                None   
3       24.0            

###  Generating the n=10,000 Test Sets (PANDAS data frames) using Random Sampling w/o Replacement

Next, we will generate another series of n=10,000 dataframes that we will use to evalutate the assumptions of our models with. To enforce random sampling without replacement, we will exlcude samples in 'case_ids' from consideration in the SQLite query.

In [10]:
# Approximate time to complete this cell = 24 seconds.
n_10000_test_set_case_ids = []  # similar to 'training_case_ids'

# 
print("Generating an N=10,000 random data point test set, composed of collisions table data objects from the switrs.sqlite database file.")
print("Working...")

# Build a tuple formatted string that contains the case_id's we are interested in and leave out those we are not.
case_ids_not_in_test_set = "("  # we will close this when we append the other argument strings.
for i in n_10000_training_set_case_ids:  # append every case_id included in the training set. 
    case_ids_not_in_test_set += i  # append it to the tuple formatted sql string argument.
    case_ids_not_in_test_set += ", "  # delete this substring before closing the string.
case_ids_not_in_test_set = case_ids_not_in_test_set[:-2]  # cut off the last two chars.
case_ids_not_in_test_set += ") ORDER BY RANDOM() LIMIT 10000;"  # complete the sqlite statement.

# This sql query demonstrates the logic behind the "random sampling without replacement" scheme. 
collisions_test_set_query = "SELECT * FROM collisions WHERE case_id NOT IN " + case_ids_not_in_test_set  # exclude training set members

# Use the sql query response to populate the 'collisions_n_10000_test_set_df' data frame object.
collisions_n_10000_test_set_df = pd.read_sql_query(collisions_test_set_query, sqliteConnection)
collisions_n_10000_test_set_df.sort_values(by=['case_id'], ascending=True, kind='quicksort', na_position='last', ignore_index=False, key=None)# sort the df in ascending order of case_id

# Check our work
print("(n=10,000) data frame formed with randomly selected 'collisions' table test tuples. These case id's compose the test set.")
print("#######################################################")
print(".head() view of the 'parties_matching_10000_df' test set dataframe object:")
print(collisions_n_10000_test_set_df.head())

# populate the 'test_case_ids' list
for i in collisions_n_10000_test_set_df.index:
    n_10000_test_set_case_ids.append(collisions_n_10000_test_set_df['case_id'].values[i])
    
print("collisions_n_10000_test_set_df.head()...")
print(collisions_n_10000_test_set_df.head())

print(n_10000_test_set_case_ids)

Generating an N=10,000 random data point test set, composed of collisions table data objects from the switrs.sqlite database file.
Working...
(n=10,000) data frame formed with randomly selected 'collisions' table test tuples. These case id's compose the test set.
#######################################################
.head() view of the 'parties_matching_10000_df' test set dataframe object:
    case_id       population county_city_location county_location  \
0   9245504   10000 to 25000                 4404      santa cruz   
1  90639966          >250000                 1942     los angeles   
2   8601739          >250000                 3313       riverside   
3   9269792   10000 to 25000                 3402      sacramento   
4   5368853  50000 to 100000                 3022          orange   

                         primary_road secondary_road  distance direction  \
0                    SCOTTS VALLEY DR   MT HERMON RD       0.0      None   
1  I-110 N/B (HAF) TO US101 (HOF/SAF) 

In [12]:
# Now we need to build the other two test set data frames.
test_set_case_ids = "("  # we will close this when we append the other argument strings.
for i in n_10000_test_set_case_ids:
    test_set_case_ids += i 
    test_set_case_ids += ", "  # delete this substring before closing the string.
test_set_case_ids = test_set_case_ids[:-2]  # 
test_set_case_ids += ") ORDER BY case_id DESC;"

# Complete each of the sql queries
parties_query = "SELECT * FROM parties WHERE case_id IN " + test_set_case_ids
victims_query = "SELECT * FROM victims WHERE case_id IN " + test_set_case_ids

collisions_n_10000_test_set_df

# PULL IN THE CORRESPONDING RECORDS FROM 'parties' and 'victims' TABLES
parties_matching_n_10000_test_set_df = pd.read_sql_query(parties_query, sqliteConnection)
victims_matching_n_10000_test_set_df = pd.read_sql_query(victims_query, sqliteConnection)

# Check our work
print("(n=10,000) data frame composed of 'parties' table tuples, associated with 'case_id's found in 'collisions_random_10000_df'")
print("#######################################################")
print(".head() view of the 'parties_matching_n_10000_test_set_df' dataframe object:")
print(parties_matching_n_10000_test_set_df.head())
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print()
print("(n=10,000) data frame composed of 'victims' table tuples, associated with 'case_id's found in 'collisions_random_10000_df'")
print("#######################################################")
print(".head() view of the 'victims_matching_n_10000_test_set_df' dataframe object:")
print(victims_matching_n_10000_test_set_df.head())

(n=10,000) data frame composed of 'parties' table tuples, associated with 'case_id's found in 'collisions_random_10000_df'
#######################################################
.head() view of the 'parties_matching_n_10000_test_set_df' dataframe object:
         id  case_id  party_number party_type  at_fault party_sex  party_age  \
0  10493099  9270809             2     driver         0      male       69.0   
1  10493098  9270809             1     driver         1      male       22.0   
2  10492953  9269792             2     driver         0    female       16.0   
3  10492952  9269792             1  bicyclist         1      male       84.0   
4  10492756  9269681             2     driver         0      male       53.0   

          party_sobriety party_drug_physical direction_of_travel  ...  \
0  had not been drinking                None                east  ...   
1  had not been drinking                None                west  ...   
2  had not been drinking                None

### Eliminating the Irrelevant Attributes

Use the cells below to evaluate the attribute columns of each table to smooth, normalize, and/or mark attribute columns for removal.

#### Examining collisions_n_10000_training_set_df

In [13]:
# TAKE A CLOSE LOOK AT THE collisions_n_10000_training_set_df DATAFRAME OBJECT (COLUMNS MANAGED WITH SQL)
print("BEGIN collisions_n_10000_training_set_df.info()")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(collisions_n_10000_training_set_df.info())
print("END collisions_n_10000_training_set_df.info()")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("BEGIN collisions_n_10000_training_set_df.head()")
print(parties_matching_10000_training_df.head())

# 
print("###################################################################################")
columns = list(collisions_n_10000_training_set_df.columns)
print("List of remaining columns in collisions_n_10000_training_set_df:")
print(columns)


BEGIN collisions_n_10000_training_set_df.info()
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 49 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   case_id                          10000 non-null  object 
 1   population                       9999 non-null   object 
 2   county_city_location             10000 non-null  object 
 3   county_location                  10000 non-null  object 
 4   primary_road                     10000 non-null  object 
 5   secondary_road                   10000 non-null  object 
 6   distance                         10000 non-null  float64
 7   direction                        7488 non-null   object 
 8   intersection                     9888 non-null   float64
 9   weather_1                        9949 non-null   object 
 10  weather_2  

### Address NULL Collision Values

In [14]:
# Start with Numeric Attributes
numeric_collision_columns = collisions_n_10000_training_set_df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_collision_columns:
    print("Column:", col)
    print(collisions_n_10000_training_set_df[col].value_counts())
    print("\n")
# Most of the numeric columns can have 0 filled in for their NaN values. Lets deal with special cases first.

# PARTY COUNT: The most common and intuitive value for this will be 2. 
collisions_n_10000_training_set_df['party_count'] = collisions_n_10000_training_set_df['party_count'].fillna(2)
# For the rest of the columns, it makes most sense to fill NaN with 0. 
# Note: filling Latitude and Longitude with 0 doesnt make much logical sense, but for the sake of avoiding errors when using them for calulations, we will fill with 0.
# If using Lat and Long in the future, consider excluding entries with 0 value.
for col in numeric_collision_columns:
    collisions_n_10000_training_set_df[col] = collisions_n_10000_training_set_df[col].fillna(0)

Column: distance
distance
0.0       2548
100.0      321
200.0      309
528.0      244
300.0      238
          ... 
516.0        1
432.0        1
291.0        1
1199.0       1
752.0        1
Name: count, Length: 863, dtype: int64


Column: intersection
intersection
0.0    7565
1.0    2323
Name: count, dtype: int64


Column: tow_away
tow_away
1.0    4991
0.0    4932
Name: count, dtype: int64


Column: killed_victims
killed_victims
0.0    9932
1.0      63
2.0       2
3.0       1
Name: count, dtype: int64


Column: injured_victims
injured_victims
0.0     6087
1.0     2758
2.0      766
3.0      242
4.0       98
5.0       32
6.0        7
7.0        2
9.0        2
12.0       1
8.0        1
Name: count, dtype: int64


Column: party_count
party_count
2    6745
1    1925
3    1035
4     226
5      44
6      16
7       6
8       3
Name: count, dtype: int64


Column: pedestrian_collision
pedestrian_collision
0    9714
1     286
Name: count, dtype: int64


Column: bicycle_collision
bicycle_collisi

In [16]:
# EXAMINE THE DATA SET
print("BEGIN: Examine collisions_n_10000_training_set_df.loc[:,'collision_date']")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(collisions_n_10000_training_set_df.loc[:,"collision_date"])
print()
print("BEGIN: Examine collisions_n_10000_training_set_df.loc[:,'ramp_intersection']")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(collisions_n_10000_training_set_df.loc[:,"ramp_intersection"])

# List of safe columns to drop: [INCLUDE ATTRIBUTE COLUMNS TO DROP HERE] 

BEGIN: Examine collisions_n_10000_training_set_df.loc[:,'collision_date']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0       2006-03-21
1       2004-08-13
2       2007-04-24
3       2016-11-16
4       2020-03-26
           ...    
9995    2001-09-06
9996    2016-05-31
9997    2009-10-12
9998    2001-03-07
9999    2009-02-05
Name: collision_date, Length: 10000, dtype: object

BEGIN: Examine collisions_n_10000_training_set_df.loc[:,'ramp_intersection']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0                                                   None
1                                                   None
2       not state highway, ramp-related, within 100 feet
3                                                   None
4                                                   None
                              ...                       
9995                                   not state highway
9996                                     

#### Examining the 'parties' Table Using 'parties_matching_10000_training_df'

In [17]:
# TAKE A CLOSE LOOK AT THE 'parties_matching_10000_training_df' DATAFRAME OBJECT (COLUMNS MANAGED WITH SQL)
print("BEGIN: Examine 'parties_matching_10000_training_df' df using '.info()'")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(parties_matching_10000_training_df.info())
print("END: parties_matching_10000_training_df.info()")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("BEGIN parties_matching_10000_training_df.head()")
print(parties_matching_10000_training_df.head())

# 
print("###################################################################################")
columns = list(parties_matching_10000_training_df.columns)
print("List of remaining columns in parties_matching_10000_training_df:")
print(columns)

BEGIN: Examine 'parties_matching_10000_training_df' df using '.info()'
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17553 entries, 0 to 17552
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            17553 non-null  int64  
 1   case_id                       17553 non-null  object 
 2   party_number                  17553 non-null  int64  
 3   party_type                    17533 non-null  object 
 4   at_fault                      17553 non-null  int64  
 5   party_sex                     15150 non-null  object 
 6   party_age                     14766 non-null  float64
 7   party_sobriety                17072 non-null  object 
 8   party_drug_physical           2984 non-null   object 
 9   direction_of_travel           17065 non-null  object 
 10  party_safety_equipment_

### Address NULL Party Values

In [18]:
# Start with Numeric Attributes
numeric_parties_columns = parties_matching_10000_training_df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_parties_columns:
    print("Column:", col)
    print(parties_matching_10000_training_df[col].value_counts())
    print("\n")
# Most of the numeric columns can have 0 filled in for their NaN values. Lets deal with special cases first.
# ID: lets just leave this alone
# PARTY NUMBER: leave this alone
# PARTY AGE: Lets fill with missing values with the mean party age
mean_age = parties_matching_10000_training_df['party_age'].mean()
parties_matching_10000_training_df['party_age'].fillna(mean_age, inplace=True)
# For the rest of the columns, it makes most sense to fill NaN with 0. 
# Note: dont touch ID or PARTY NUMBER as these could create unintentional results and it wont be needed for calculations

# Get all column names except 'id' and 'party_number'
parties_cols_to_fill = [col for col in parties_matching_10000_training_df.columns if col not in ['id', 'party_number']]

# Fill missing values in these columns with 0
parties_matching_10000_training_df[parties_cols_to_fill] = parties_matching_10000_training_df[parties_cols_to_fill].fillna(0)

Column: id
id
1877320    2
5251184    2
4689872    2
8907703    2
2924482    2
          ..
4963611    1
4963030    1
4963029    1
4962128    1
1798930    1
Name: count, Length: 17547, dtype: int64


Column: party_number
party_number
1    8859
2    7156
3    1185
4     261
5      60
6      22
7       8
8       2
Name: count, dtype: int64


Column: at_fault
at_fault
0    9417
1    8136
Name: count, dtype: int64


Column: party_age
party_age
22.0    458
19.0    444
20.0    440
18.0    430
23.0    429
       ... 
89.0      2
2.0       1
93.0      1
98.0      1
99.0      1
Name: count, Length: 97, dtype: int64


Column: cellphone_in_use
cellphone_in_use
0.0    13030
1.0      271
Name: count, dtype: int64


Column: school_bus_related
school_bus_related
1.0    56
Name: count, dtype: int64


Column: party_number_killed
party_number_killed
0    17495
1       56
3        1
2        1
Name: count, dtype: int64


Column: party_number_injured
party_number_injured
0     13337
1      3593
2       46

In [19]:
# Identify the numeric columns fo the parties training set dataframe.
parties_cols_to_scale = parties_matching_10000_training_df.select_dtypes(include=['float64', 'int64']).columns
parties_cols_to_scale = [col for col in parties_cols_to_scale if col not in ['id', 'party_number']]
print("parties_cols_to_scale", parties_cols_to_scale)

parties_cols_to_scale ['at_fault', 'party_age', 'cellphone_in_use', 'school_bus_related', 'party_number_killed', 'party_number_injured', 'vehicle_year']


#### Examinine the quality of data cleaning and smoothing.

### Address NULL Victim Values

In [20]:
# Start with Numeric Attributes
numeric_victims_columns = victims_matching_10000_training_df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_victims_columns:
    print("Column:", col)
    print(victims_matching_10000_training_df[col].value_counts())
    print("\n")

# ID: lets just leave this alone
# VICTIM NUMBER: leave this alone
# VICTIM AGE: Lets fill with missing values with the mean party age
mean_age = victims_matching_10000_training_df['victim_age'].mean()
victims_matching_10000_training_df['victim_age'].fillna(mean_age, inplace=True)
# There are no more numeric attributes
# Note: dont touch ID or PARTY NUMBER as these could create unintentional results and it wont be needed for calculations

Column: id
id
278150     2
5078102    1
145051     1
151411     1
151412     1
          ..
2350582    1
2352109    1
2356260    1
2356261    1
1029848    1
Name: count, Length: 8811, dtype: int64


Column: party_number
party_number
2    4154
1    4005
3     541
4      74
5      30
6       6
7       1
8       1
Name: count, dtype: int64


Column: victim_age
victim_age
19.0     272
18.0     262
20.0     252
22.0     230
23.0     230
        ... 
92.0       1
98.0       1
96.0       1
102.0      1
94.0       1
Name: count, Length: 98, dtype: int64




In [21]:
# making df to scale because we want to exclude id and party_number
victim_cols_to_scale = victims_matching_10000_training_df.select_dtypes(include=['float64', 'int64']).columns
victim_cols_to_scale = [col for col in victim_cols_to_scale if col not in ['id', 'party_number']]

### Examine the resulting attribute outcomes.

In [22]:
# OBSERVE THE EFFECTS OF DATA CLEANING AND DATA SMOOTHING ON THE TRAINING SET DATA FRAMES:

# # USE THE CODE BELOW TO EXPLORE INTERESTING ATTRIBUTES IN ANY OF THE AVAILABLE DATAFRAMES.
# print("BEGIN: Examine parties_matching_10000_training_df.loc[:,'movement_preceding_collision']")
# print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# print(parties_matching_10000_training_df.loc[:,"movement_preceding_collision"])  # interesting attribute
# print()
# print("BEGIN: Examine parties_matching_10000_training_df.loc[:,'cellphone_in_use']")
# print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# print(parties_matching_10000_training_df.loc[:,"cellphone_in_use"])


# SUMMARIZE DATA FRAMES WITH '.info()'
# collisions_n_10000_training_set_df.info, collisions_n_10000_training_set_df.head()
print("BEGIN: Examine 'collisions_n_10000_training_set_df' df using '.info()'")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(collisions_n_10000_training_set_df.info())
print("END collisions_n_10000_training_set_df.info()")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("BEGIN collisions_n_10000_training_set_df.head()")
print(collisions_n_10000_training_set_df.head())

# parties_matching_10000_training_df.info(), parties_matching_10000_training_df.head()
print("BEGIN: Examine 'parties_matching_10000_training_df' df using '.info()'")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(parties_matching_10000_training_df.info())
print("END parties_matching_10000_training_df.info()")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("BEGIN parties_matching_10000_training_df.head()")
print(parties_matching_10000_training_df.head())

# victims_matching_10000_training_df.info(), victims_matching_10000_training_df.head()
print("BEGIN: Examine 'victims_matching_10000_training_df' df using '.info()'")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(victims_matching_10000_training_df.info())
print("END victims_matching_10000_training_df.info()")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("BEGIN victims_matching_10000_training_df.head()")
print(victims_matching_10000_training_df.head())

# 
# print("###################################################################################")
# columns = list(victims_matching_10000_training_df.columns)
# print("List of remaining columns in victims_matching_10000_training_df:")
# print(columns)

BEGIN: Examine 'collisions_n_10000_training_set_df' df using '.info()'
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 49 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   case_id                          10000 non-null  object 
 1   population                       9999 non-null   object 
 2   county_city_location             10000 non-null  object 
 3   county_location                  10000 non-null  object 
 4   primary_road                     10000 non-null  object 
 5   secondary_road                   10000 non-null  object 
 6   distance                         10000 non-null  float64
 7   direction                        7488 non-null   object 
 8   intersection                     10000 non-null  float64
 9   weather_1                        9949 non-null   o

In [23]:
# OBSERVE NOMINAL ATTRIBUTES OF EACH TABLE + UNIQUE NOMINAL ATTRIBUTE OUTCOMES IN EACH TABLE
nominal_collisions_columns = collisions_n_10000_training_set_df.select_dtypes(include=['object']).columns
print("Nominal attributes of the collisions table:")
print(nominal_collisions_columns)
print()
print("Nominal attributes of the parties table:")
nominal_parties_columns = parties_matching_10000_training_df.select_dtypes(include=['object']).columns
print(nominal_parties_columns)
print()
print("Nominal attributes of the victims table:")
nominal_victims_columns = victims_matching_10000_training_df.select_dtypes(include=['object']).columns
print(nominal_victims_columns)
print()


# PRINT UNIQUE NOMINAL ATTRIBUTE OUTCOMES
# Examine unique outcomes in 'collisions_n_10000_training_set_df'
print("collisions_n_10000_training_set_df unique nominal attribute outcomes after data cleaning:")
for col in collisions_n_10000_training_set_df[nominal_collisions_columns]:
    print(collisions_n_10000_training_set_df[col].unique())
print()
# Examine unique outcomes in 'parties_matching_10000_training_df'
print("parties_matching_10000_training_df unique nominal attribute outcomes after data cleaning:")
for col in parties_matching_10000_training_df[nominal_parties_columns]:
    print(parties_matching_10000_training_df[col].unique())
print()
# Examine unique outcomes in 'victims_matching_10000_training_df'
print("victims_matching_10000_training_df unique nominal attribute outcomes after data cleaning:")
for col in victims_matching_10000_training_df[nominal_victims_columns]:
    print(victims_matching_10000_training_df[col].unique())
print()


# EXAMINE NUMERIC ATTRIBUTES AFTER DATA CLEANING
print("collisions_n_10000_training_set_df statistics after data cleaning:")
print(collisions_n_10000_training_set_df[numeric_collision_columns].describe().round(2))  # collisions training set
print()
print("parties_matching_10000_training_df statistics after data cleaning:")
print(parties_matching_10000_training_df[numeric_parties_columns].describe().round(2))  # parties training set
print()
print("victims_matching_10000_training_df statistics after data cleaning:")
print(victims_matching_10000_training_df.describe().round(2))  # victims training set 

Nominal attributes of the collisions table:
Index(['case_id', 'population', 'county_city_location', 'county_location',
       'primary_road', 'secondary_road', 'direction', 'weather_1', 'weather_2',
       'location_type', 'ramp_intersection', 'collision_severity',
       'primary_collision_factor', 'pcf_violation_code',
       'pcf_violation_category', 'type_of_collision',
       'motor_vehicle_involved_with', 'pedestrian_action', 'road_surface',
       'road_condition_1', 'lighting', 'control_device',
       'statewide_vehicle_type_at_fault', 'chp_vehicle_type_at_fault',
       'collision_date', 'collision_time'],
      dtype='object')

Nominal attributes of the parties table:
Index(['case_id', 'party_type', 'party_sex', 'party_sobriety',
       'party_drug_physical', 'direction_of_travel',
       'party_safety_equipment_1', 'party_safety_equipment_2',
       'financial_responsibility', 'cellphone_use_type',
       'oaf_violation_category', 'oaf_violation_section',
       'oaf_violat

In [24]:
# Let's take a look at the categorical column distributions next...
print("statistics on the nominal attributes of the collisions_n_10000_training_set_df data set after cleaning:")
print(collisions_n_10000_training_set_df[nominal_collisions_columns].describe())  # collisions training set
print()
print("statistics on the nominal attributes of the parties_matching_10000_training_df data set after cleaning:")
print(parties_matching_10000_training_df[nominal_parties_columns].describe())  # parties training set
print()
print("statistics on the nominal attributes of the victims_matching_10000_training_df data set after cleaning:")
print(victims_matching_10000_training_df[nominal_victims_columns].describe())  # victims training set 

statistics on the nominal attributes of the collisions_n_10000_training_set_df data set after cleaning:
        case_id population county_city_location county_location primary_road  \
count     10000       9999                10000           10000        10000   
unique    10000          8                  471              58         4650   
top     2584916    >250000                 1942     los angeles       RT 101   
freq          1       2503                 1164            2981          237   

       secondary_road direction weather_1 weather_2 location_type  ...  \
count           10000      7488      9949       274          2766  ...   
unique           6615         4         7         6             3  ...   
top           MAIN ST     south     clear   raining       highway  ...   
freq               39      2061      8228       194          2140  ...   

       motor_vehicle_involved_with       pedestrian_action road_surface  \
count                         9955               

### Numeric Principle Component Analysis (PCA)

In [25]:
import numpy as np
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler

# combine the numeric attributes of the training tables to perform rigorous numerical PCA
df_collision_parties = pd.merge(collisions_n_10000_training_set_df, parties_matching_10000_training_df, how='inner')  # inner-join training sets for 'collisions' and 'parties' tables
df_collision_victims = pd.merge(collisions_n_10000_training_set_df, victims_matching_10000_training_df, how='inner')  # inner-join training sets for 'collisions' and 'victims' tables
# print(df_collision_parties.info())  # for debugging 
# print(df_collision_victims.info())  # for debugging 

# next, we need to union the attribute indices of the new dataframes to form a new combined index.
collision_parties_numeric_index = numeric_collision_columns.union(numeric_parties_columns)  # union 'numeric_collision_columns' and 'numeric_parties_columns'
collision_victims_numeric_index = numeric_collision_columns.union(numeric_victims_columns)  # union 'numeric_collision_columns' and 'numeric_parties_columns'
# print(df_collision_parties[collision_parties_index].head())  # for debugging 
# print(df_collision_victims[collision_victims_index].head())  # for debugging 

In [26]:
# POTENTIAL FORMAT FOR FUTURE CLASSIFICATION MODELS:
# X, y =   df_collision_parties[collision_parties_numeric_index], df_collision_parties[collision_parties_numeric_index] # assign two numeric columns
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages 
# from pypdf import PdfReader


# Perform Numerical PCA on 'df_collision_parties'
X = df_collision_parties[collision_parties_numeric_index]
# y = df_collision_parties['primary_collision_factor']  # Attribute whose outcome we would like to classify future data points with.

min_max_scaler = MinMaxScaler()  # min_max_scaler object
std_scaler = StandardScaler()  # std_scaler object
pca = PCA(n_components=len(collision_parties_numeric_index))  # match the number of attribute columns under examination.
# pca = PCA(n_components=4)  # shortened version for testing.


# USE THIS CODE TO APPLY THE TRANSFORMATION TO THE DATA FRAME
# X_pca = pca.fit_transform(df_collision_parties[collision_parties_numeric_index])  # Perform PCA "as-is"
# X_pca = pca.fit_transform(min_max_scaler.fit_transform(df_collision_parties[collision_parties_numeric_index]))  # apply min_max transform
X_pca = pca.fit_transform(std_scaler.fit_transform(df_collision_parties[collision_parties_numeric_index]))  # apply standard (z-score) transform
  
  # create the pdf file that will store the Numerical PCA figures
pdf_file = PdfPages('./df_collision_parties_std_scaler_numerical_pca.pdf')  # identify the write-to object
            
fig_count = 1  # keep a rolling count of the figures generated so far.
# WARNING: This code will generate a LOT of graphs! 
print("BEGIN NUMERICAL PCA ON 'df_collision_parties'")
for i in range(len(collision_parties_numeric_index)):  # shorten to test the algorithm.
# for i in range(4):  # shorten to test the algorithm.  # shortened version for testing.
    for j in range(len(collision_parties_numeric_index)):
    # for j in range(4):  # shortened version for testing.
        if i != j: 
            title = "Numerical PCA:" + "'" + collision_parties_numeric_index[i] + "'" + " vs. " + "'" + collision_parties_numeric_index[j] + "'"
            plt.title(title)
            plt.scatter(X_pca[:, i], X_pca[:, j])  # change the values at these index definitions to control which variables are compared.
            # plt.xlabel(collision_parties_numeric_index[i] + " (as-is)")  # replace these labels with the column name
            # plt.ylabel(collision_parties_numeric_index[j] + " (as-is)")
            # plt.xlabel(collision_parties_numeric_index[i] + " (min-max normalized)")  # replace these labels with the column name
            # plt.ylabel(collision_parties_numeric_index[j] + " (min-max normalized)")
            plt.xlabel(collision_parties_numeric_index[i] + " (z-score normalized)")  # replace these labels with the column name
            plt.ylabel(collision_parties_numeric_index[j] + " (z-score normalized)")
            
            # plt.show()  # TODO: replace .show() statement with a repeated write to a specified external pdf file.
            plt.savefig(pdf_file, format='pdf')  # append each figure to the indicated .pdf file.
            print(title, "is complete.")
            print("Figure count so far:", fig_count)
            plt.clf()  # clear the figure each time or the plots get overlaid on every subsequent graph.
            fig_count += 1
            
pdf_file.close()  # close the pdf file when we are done.
print("See 'df_collision_parties_std_scaler_numerical_pca.pdf' for results!")

BEGIN NUMERICAL PCA ON 'df_collision_parties'
Numerical PCA:'alcohol_involved' vs. 'at_fault' is complete.
Figure count so far: 1
Numerical PCA:'alcohol_involved' vs. 'bicycle_collision' is complete.
Figure count so far: 2
Numerical PCA:'alcohol_involved' vs. 'bicyclist_injured_count' is complete.
Figure count so far: 3
Numerical PCA:'alcohol_involved' vs. 'bicyclist_killed_count' is complete.
Figure count so far: 4
Numerical PCA:'alcohol_involved' vs. 'cellphone_in_use' is complete.
Figure count so far: 5
Numerical PCA:'alcohol_involved' vs. 'complaint_of_pain_injury_count' is complete.
Figure count so far: 6
Numerical PCA:'alcohol_involved' vs. 'distance' is complete.
Figure count so far: 7
Numerical PCA:'alcohol_involved' vs. 'id' is complete.
Figure count so far: 8
Numerical PCA:'alcohol_involved' vs. 'injured_victims' is complete.
Figure count so far: 9
Numerical PCA:'alcohol_involved' vs. 'intersection' is complete.
Figure count so far: 10
Numerical PCA:'alcohol_involved' vs. 'ki

<Figure size 640x480 with 0 Axes>

In [27]:
# POTENTIAL FORMAT FOR FUTURE CLASSIFICATION MODELS:
# X, y =   df_collision_parties[collision_parties_numeric_index], df_collision_parties[collision_parties_numeric_index] # assign two numeric columns
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages 
# from pypdf import PdfReader


# Perform Numerical PCA on 'df_collision_victims'
X = df_collision_victims[collision_victims_numeric_index]
# y = df_collision_parties['primary_collision_factor']  # Attribute whose outcome we would like to classify future data points with.

min_max_scaler = MinMaxScaler()
pca = PCA(n_components=len(collision_victims_numeric_index))  # match the number of attribute columns under examination.
# pca = PCA(n_components=4)  # shortened version for testing.


# USE THIS CODE TO APPLY THE TRANSFORMATION TO THE DATA FRAME
# X_pca = pca.fit_transform(df_collision_victims[collision_victims_numeric_index])  # Perform PCA "as-is"
# X_pca = pca.fit_transform(min_max_scaler.fit_transform(df_collision_victims[collision_victims_numeric_index]))  # apply min_max transform
X_pca = pca.fit_transform(std_scaler.fit_transform(df_collision_victims[collision_victims_numeric_index]))  # apply standard (z-score) transform

# create the pdf file that will store the Numerical PCA figures
pdf_file = PdfPages('./df_collision_victims_std_scaler_numerical_pca.pdf')  # identify the write-to object
            
fig_count = 1  # keep a rolling count of the figures generated so far.
# WARNING: This code will generate a LOT of graphs! 
print("BEGIN NUMERICAL PCA ON 'df_collision_victims'")
for i in range(len(collision_victims_numeric_index)):  # shorten to test the algorithm.
# for i in range(4):  # shorten to test the algorithm.  # shortened version for testing.
    for j in range(len(collision_victims_numeric_index)):
    # for j in range(4):  # shortened version for testing.
        if i != j: 
            title = "Numerical PCA:" + "'" + collision_victims_numeric_index[i] + "'" + " vs. " + "'" + collision_victims_numeric_index[j] + "'"
            plt.title(title)
            plt.scatter(X_pca[:, i], X_pca[:, j])  # change the values at these index definitions to control which variables are compared.
            # plt.xlabel(collision_victims_numeric_index[i] + " (as-is)")  # replace these labels with the column name
            # plt.ylabel(collision_victims_numeric_index[j] + " (as-is)")
            # plt.xlabel(collision_victims_numeric_index[i] + " (min-max normalized)")  # replace these labels with the column name
            # plt.ylabel(collision_victims_numeric_index[j] + " (min-max normalized)")
            plt.xlabel(collision_victims_numeric_index[i] + " (z-score normalized)")  # replace these labels with the column name
            plt.ylabel(collision_victims_numeric_index[j] + " (z-score normalized)")
            
            # plt.show()  # TODO: replace .show() statement with a repeated write to a specified external pdf file.
            plt.savefig(pdf_file, format='pdf')  # append each figure to the indicated .pdf file.
            print(title, "is complete.")
            print("Figure count so far:", fig_count)
            plt.clf()  # clear the figure each time or the plots get overlaid on every subsequent graph.
            fig_count += 1
            
pdf_file.close()  # close the pdf file when we are done.
print("See 'df_collision_victims_std_scaler_numerical_pca.pdf' for results!")

BEGIN NUMERICAL PCA ON 'df_collision_victims'
Numerical PCA:'alcohol_involved' vs. 'bicycle_collision' is complete.
Figure count so far: 1
Numerical PCA:'alcohol_involved' vs. 'bicyclist_injured_count' is complete.
Figure count so far: 2
Numerical PCA:'alcohol_involved' vs. 'bicyclist_killed_count' is complete.
Figure count so far: 3
Numerical PCA:'alcohol_involved' vs. 'complaint_of_pain_injury_count' is complete.
Figure count so far: 4
Numerical PCA:'alcohol_involved' vs. 'distance' is complete.
Figure count so far: 5
Numerical PCA:'alcohol_involved' vs. 'id' is complete.
Figure count so far: 6
Numerical PCA:'alcohol_involved' vs. 'injured_victims' is complete.
Figure count so far: 7
Numerical PCA:'alcohol_involved' vs. 'intersection' is complete.
Figure count so far: 8
Numerical PCA:'alcohol_involved' vs. 'killed_victims' is complete.
Figure count so far: 9
Numerical PCA:'alcohol_involved' vs. 'latitude' is complete.
Figure count so far: 10
Numerical PCA:'alcohol_involved' vs. 'long

<Figure size 640x480 with 0 Axes>

### Categorical PCA

The following runs Chi2 test on all pairs of categorical variables in the df_collision_parties dataframe and saves the results that show a potential association based on p value in a pdf 

In [34]:
from scipy.stats import chi2_contingency

# Identify categorical attributes of type object
object_cols = df_collision_parties.select_dtypes(include='object').columns

# Initialize results dictionary
collision_parties_chi2_results = {}

# Perform chi-squared test on every pair of object columns using nested loops
for i in range(len(object_cols)):
    for j in range(i+1, len(object_cols)):
        col1 = object_cols[i]
        col2 = object_cols[j]
        
        contingency = pd.crosstab(df_collision_parties[col1], df_collision_parties[col2])
        chi2, p, _, _ = chi2_contingency(contingency)
        
        # Store results if potential association
        if p <= 0.05 and p > 0.0001:
            collision_parties_chi2_results[(col1, col2)] = {'chi2': chi2, 'p_value': p}

# Save results to a PDF using matplotlib
with plt.ioff():  # Turn interactive plotting off
    fig, ax = plt.subplots(figsize=(10, len(collision_parties_chi2_results) * 1.5))
    ax.axis('off')
    
    results_text = ''
    for pair, results in collision_parties_chi2_results.items():
        results_text += f"Columns: {pair}\nChi-squared: {results['chi2']:.4f}, p-value: {results['p_value']:.4f}\n------\n"
    
    ax.text(0, 1, results_text, verticalalignment='top', fontsize=12)
    
    plt.savefig("df_collision_parties_chi2_results.pdf", bbox_inches='tight')
    plt.close(fig)

This does the same as above, but for df_collision_victims instead

In [36]:
# Identify columns of type object
object_cols = df_collision_victims.select_dtypes(include='object').columns

# Initialize results dictionary
collision_victims_chi2_results = {}

# Perform chi-squared test on every pair of object columns using nested loops
for i in range(len(object_cols)):
    for j in range(i+1, len(object_cols)):
        col1 = object_cols[i]
        col2 = object_cols[j]
        
        if df_collision_victims[col1].notna().any() and df_collision_victims[col2].notna().any():
            contingency = pd.crosstab(df_collision_victims[col1], df_collision_victims[col2])
            
            if not contingency.empty:
                chi2, p, _, _ = chi2_contingency(contingency)
                
                # Store results if potential association
                if p <= 0.05 and p > 0.0001:
                    collision_victims_chi2_results[(col1, col2)] = {'chi2': chi2, 'p_value': p}

# Save results to a PDF using matplotlib
with plt.ioff():  # Turn interactive plotting off
    fig, ax = plt.subplots(figsize=(10, len(collision_victims_chi2_results) * 1.5))
    ax.axis('off')
    
    results_text = ''
    for pair, results in collision_victims_chi2_results.items():
        results_text += f"Columns: {pair}\nChi-squared: {results['chi2']:.4f}, p-value: {results['p_value']:.4f}\n------\n"
    
    ax.text(0, 1, results_text, verticalalignment='top', fontsize=12)
    
    plt.savefig("df_collision_victims_chi2_results.pdf", bbox_inches='tight')
    plt.close(fig)

Below, we will create a DataFrame containing attributes that are most intuitively associated with collisions. We can also print out a row to see what the data looks like.

In [83]:
collisions_important_df = collisions_random_10000_df[['intersection', 'state_highway_indicator', 'killed_victims', 
    'injured_victims', 'party_count', 'pedestrian_collision', 'bicycle_collision', 'motorcycle_collision', 
    'truck_collision', 'alcohol_involved', 'population', 'weather_1', 'location_type', 'collision_severity', 
    'primary_collision_factor', 'type_of_collision', 'pedestrian_action', 'road_surface', 'road_condition_1', 
    'lighting', 'control_device']]

collisions_important_df['alcohol_involved'] = collisions_important_df['alcohol_involved'].fillna(0) # Filling in 0 for NaN values in alcohol involvement so they can be used in calculations.

print(collisions_important_df.head(1)) # Print one row of the table so we can see what it looks like

   intersection  state_highway_indicator  killed_victims  injured_victims  \
0           1.0                      1.0             0.0              0.0   

   party_count  pedestrian_collision  bicycle_collision  motorcycle_collision  \
0            2                     0                  0                     0   

   truck_collision  alcohol_involved  ... weather_1 location_type  \
0                0               0.0  ...     clear          ramp   

     collision_severity primary_collision_factor type_of_collision  \
0  property damage only   vehicle code violation         sideswipe   

        pedestrian_action road_surface road_condition_1  \
0  no pedestrian involved          dry           normal   

                  lighting control_device  
0  dark with street lights    functioning  

[1 rows x 21 columns]


C:\Users\jmica\AppData\Local\Temp\ipykernel_27400\3099857728.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collisions_important_df['alcohol_involved'] = collisions_important_df['alcohol_involved'].fillna(0) # Filling in 0 for NaN values in alcohol involvement so they can be used in calculations.


In [ ]:
# This cell will defines a first round Data Cube Function(s)
create_data_cube = """SELECT 
    c1, c2, AGGREGATE_FUNCTION(c3)
FROM
    table_name
GROUP BY CUBE(c1 , c2);"""